In [58]:
###################################
#                                 #
#  CTU_efficiency_statistics      #
#  Akhil Garg, akhil@akhilgarg.ca #
#  Created 2021-04-22             #
#                                 #
###################################

import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates  as mdates
import scipy.stats       as stats
import openpyxl


CTU_data      = pd.read_pickle('CTU_data.pickle')
CCI           = pd.read_excel ('Charlson Comorbidity Index.xlsx')
date_frame    = pd.read_pickle('Daily CTU Census.pickle')

## Data cleaning

In [59]:
# Only keep data before March 15, 2020, before COVID really hit
CTU_data = CTU_data[CTU_data['ADMISSION_DATE_TIME']<=pd.to_datetime('2020-03-15')]

In [71]:
# Merge CCI data into dataframe
all_data = CTU_data.merge(CCI[['daily_admits','Drip?','G_exists?','CCI']],
               how='left',left_index=True,right_index=True)

column_order = ['ConsultRequestDateTime','ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME',
                'SEX','AGE','CCI','daily_admits','Drip?','G_exists?',
                'consult_duration_hours','Death','Readmission','LOS']

all_data = all_data[column_order]

# Rename columns to remove special characters and spaces
all_data = all_data.rename({'Drip?':'Drip','G_exists?':'G_exists'},axis='columns')

# Sort data
all_data = all_data.sort_values(by='ADMISSION_DATE_TIME')

In [72]:
# Add a column for daily admissions
def admit_day_8(date):
    '''
    Given a pandas datetime object
    Returns what day the datetime object corresponds to
    Dates between midnight and 8 am count for the previous day
    '''
    
    # Admissions from 08 to 23 count for the current day
    if date.hour >= 8:
        admit_day = pd.to_datetime(
            str(admission.ADMISSION_DATE_TIME.date())+' 08:00')
        
    # Admissions after midnight (00-07) are counted for the previous day
    elif date.hour <8:
        admit_day = pd.to_datetime(
            str(admission.ADMISSION_DATE_TIME.date()-pd.Timedelta('1d'))
            +' 08:00')
        
    return admit_day

# Reset daily admits
all_data['daily_admits'] = 0

# Create temporary dataframes, 
# as looping over an existing dataframe while making edits
# can lead to unexpected behaviour
temp_df = all_data

# Number of admissions per day
daily_admits = date_frame.groupby('date')['admits'].sum()

for admission in all_data.itertuples():
                
    temp_df.loc[admission.Index,'daily_admits'] = \
    daily_admits.loc[admit_day_8(admission.ADMISSION_DATE_TIME)]

In [73]:
all_data

,ConsultRequestDateTime,ADMISSION_DATE_TIME,DISCHARGE_DATE_TIME,SEX,AGE,CCI,daily_admits,Drip,G_exists,consult_duration_hours,Death,Readmission,LOS
0,2015-07-01 13:02:00,2015-07-01 16:58:00,2015-07-03 16:57:00,Female,86.166320,2,7.0,0,0,3.933333,False,False,1.999306
1,2015-07-01 16:09:00,2015-07-01 18:57:00,2015-08-01 11:38:00,Male,66.667917,2,7.0,0,0,2.800000,False,False,30.695139
2,2015-07-01 15:57:00,2015-07-01 20:01:00,2015-07-08 09:44:00,Female,71.095436,1,7.0,0,0,4.066667,False,False,6.571528
3,2015-07-01 18:51:00,2015-07-01 22:36:00,2015-07-07 14:35:00,Male,71.117648,0,7.0,0,0,3.750000,False,False,5.665972
4,2015-07-01 12:48:00,2015-07-01 23:00:00,2015-07-04 07:07:00,Female,43.531393,0,7.0,0,0,10.200000,False,False,2.338194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19137,2020-03-14 07:07:00,2020-03-14 13:43:00,2020-03-20 15:17:00,Female,80.894717,1,11.0,1,1,6.600000,False,False,6.065278
19138,2020-03-14 12:00:00,2020-03-14 16:47:00,2020-03-19 16:08:00,Female,87.960820,1,11.0,1,1,4.783333,False,False,4.972917
19139,2020-03-14 13:17:00,2020-03-14 19:20:00,2020-03-25 13:03:00,Male,67.046043,2,11.0,1,1,6.050000,False,False,10.738194
19140,2020-03-14 13:39:00,2020-03-14 22:10:00,2020-04-01 08:52:00,Female,91.084722,1,11.0,1,1,8.516667,False,False,17.445833


In [74]:
all_data.to_excel('CTU_data 2021-05-27.xlsx')

### Stats

In [64]:
print('ED consult time to admission')

# Describe means
b_mean = all_data[all_data['Drip?']==0]['consult_duration_hours'].mean()
d_mean = all_data[all_data['Drip?']==1]['consult_duration_hours'].mean()
print('\nMean')
print('Bolus: {:.2f} h'.format(b_mean))
print('Drip:  {:.2f} h'.format(d_mean))
print('Difference: {:.2f} h or {:.2f}%'.format(
    d_mean-b_mean,(d_mean-b_mean)/b_mean*100))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.2f} h'.format(
    all_data[all_data['Drip?']==0]['consult_duration_hours'].std()))
print('Drip:  {:.2f} h'.format(
    all_data[all_data['Drip?']==1]['consult_duration_hours'].std()))

# Compare means
print('\nMann-Whitney test for difference in mean')
print('P-value = {:.2e}'.format(stats.mannwhitneyu(
    x=all_data[all_data['Drip?']==0]['consult_duration_hours'],
    y=all_data[all_data['Drip?']==1]['consult_duration_hours'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance')
print('P-value = {:.2e}'.format(stats.levene
    (all_data[all_data['Drip?']==0]['consult_duration_hours'],
     all_data[all_data['Drip?']==1]['consult_duration_hours'])[1]))

ED consult time to admission

Mean
Bolus: 5.04 h
Drip:  4.78 h
Difference: -0.25 h or -5.05%

Standard deviation
Bolus: 2.52 h
Drip:  2.44 h

Mann-Whitney test for difference in mean
P-value = 1.08e-13

Levene test for difference in variance
P-value = 7.75e-04
